# NFL superbowl & mvp analysis 

## Imports

In [62]:
# Import Dependecies
import numpy as np
import datetime as dt
from datetime import datetime
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine

In [8]:
# Import Configurations

# Import DB User id & password 
from Configs import  db_user
from Configs import db_password

# Import API Key
from Configs import weather_api_key

In [9]:
# Import Functions 
from Functions import int_to_roman

## Script notes

In [71]:
# Script Notes: This scripot will:
#
# Read the csv file supoerbowl.csv in Resources directory to fetch NFL superbowl history into a Dataframe 
# Sort & Transaform the DataFrame
# Read the nfl_players.json file in Resources directory to fetch the information about NFL players into a DataFrame 
# Clean-up, Index & Tranform the Dataframe
#
# For each SupoerBowl match:
#    Fetch weather information using open weather map historical data API
#    Add weather information to the superbowl dataframe  
#
# Load the superbowl & player DataFrame to Postgres nfl_db 
# 
# load DataFrame(s) to Postgres SQL tables
# Join the tables and save as a view
# Display data from the view to confirm data loaded


# Weather API information:
# http://history.openweathermap.org/data/2.5/history/city?q={city ID},{country code}&type=hour&start={start}&end={end}
# Parameters:
# q city name and country code divided by comma, use ISO 3166 country codes 
# type type of the call, keep this parameter in the API call as 'hour'
# start start date (unix time, UTC time zone), e.g. start=1369728000
# end end date (unix time, UTC time zone), e.g. end=1369789200
# cnt amount of returned data (one per hour, can be used instead of 'end')
# Examples API call:
# http://history.openweathermap.org/data/2.5/history/city?q=London,US

## Load superbowl data from CSV file 

In [72]:
# Load the Superbowl CSV file 

# Path to Superbowl input CSV file
suberbowl_file = "../Resources/superbowl.csv"

# Load csv file to DataFrame
superbowl_df = pd.read_csv(suberbowl_file)

# Display DataFrame
superbowl_df.head()

,Date,SB,Winner,Winner Pts,Loser,Loser Pts,MVP,Stadium,City,State
0,Feb 2 2020,LIV (54),Kansas City Chiefs,31,San Francisco 49ers,20,Patrick Mahomes,Hard Rock Stadium,Miami Gardens,Florida
1,Feb 3 2019,LIII (53),New England Patriots,13,Los Angeles Rams,3,Julian Edelman,Mercedes-Benz Stadium,Atlanta,Georgia
2,Feb 4 2018,LII (52),Philadelphia Eagles,41,New England Patriots,33,Nick Foles,U.S. Bank Stadium,Minneapolis,Minnesota
3,Feb 5 2017,LI (51),New England Patriots,34,Atlanta Falcons,28,Tom Brady,NRG Stadium,Houston,Texas
4,Feb 7 2016,50,Denver Broncos,24,Carolina Panthers,10,Von Miller,Levi's Stadium,Santa Clara,California


### Data clean-up & transformation (superbowl)

In [73]:
# Cleanup & Transform the superbowl DataFrame

# Copy the Supoerbowl DataFrame to Superbowl Transformed Dataframe  
superbowl_t_df = superbowl_df.copy()

# Note: The SB column all rows do NOT have both the number and roman number for the superbowl id. 
# Establish a numeric id based on date and convert to roman number also as superbowl name

# Set a Python date for teh date in CSV  
superbowl_t_df['date'] = pd.to_datetime(superbowl_t_df['Date'])
# Sort the Dataframe based on Date 
superbowl_t_df.sort_values('date', ascending=True, inplace=True)

# Establish an ID for the Superbowl number sorted on Superbowl Date
# Start with an empty list and append to it for the number of SuperBowls 
id_list = []
id_roman_list = []
for i in range(0, len(superbowl_t_df["SB"])):
    id_list.append(int(i+1))
    id_roman_list.append("Superbowl " + str(int_to_roman(i+1)))
# Add id & name as columns to the Dataframe 
superbowl_t_df['id'] = id_list
superbowl_t_df['name'] = id_roman_list

# Add a column for score
superbowl_t_df['score'] = superbowl_t_df['Winner Pts'].astype(str) + ' - ' + superbowl_t_df['Loser Pts'].astype(str)

# Clean up the MVP Column to remove the "+" at the end
superbowl_t_df['MVP'] = superbowl_t_df['MVP'].str.replace('+', '')

# Drop columns "SB", "Date", "Winner Pts", "Loser Pts"
superbowl_t_df.drop(superbowl_t_df.columns[[0, 1, 3, 5]], axis = 1, inplace = True)

# Rename Columns to match columns in postgres table  
superbowl_t_df.rename(columns={"Winner" : "winner", "Loser" : "loser",\
                               "MVP" : "mvp", "Stadium" : "stadium",\
                               "City" : "city", "State" : "state", }, inplace=True)

# Rearrange the dataframe to match the table definition 
cols = ['id', 'name', 'date', 'winner', 'loser', 'score', 'mvp', 'city', 'state', 'stadium']
superbowl_t_df = superbowl_t_df[cols]

# Set index as id
superbowl_t_df.reset_index(inplace=True)
superbowl_t_df.set_index("id", inplace=True)

# Display the DataFrame
superbowl_t_df.head()

,index,name,date,winner,loser,score,mvp,city,state,stadium
id,,,,,,,,,,
1,53,Superbowl I,1967-01-15,Green Bay Packers,Kansas City Chiefs,35 - 10,Bart Starr,Los Angeles,California,Memorial Coliseum
2,52,Superbowl II,1968-01-14,Green Bay Packers,Oakland Raiders,33 - 14,Bart Starr,Miami,Florida,Orange Bowl
3,51,Superbowl III,1969-01-12,New York Jets,Baltimore Colts,16 - 7,Joe Namath,Miami,Florida,Orange Bowl
4,50,Superbowl IV,1970-01-11,Kansas City Chiefs,Minnesota Vikings,23 - 7,Len Dawson,New Orleans,Louisiana,Tulane Stadium
5,49,Superbowl V,1971-01-17,Baltimore Colts,Dallas Cowboys,16 - 13,Chuck Howley,Miami,Florida,Orange Bowl


### Get weather information

#### <font color='red'>Maria - Add code below</font>

In [76]:
# Maria - Add code here:


# For each row in Dataframe: 
# Convert date to Unix date format
# Get weather information using City, Date & Country = 'US'
# Perform data tranformation & clean-up 
# Add Weather information to the Dataframe (Only columns we want to save to Database table)




## Load player data from JSON file 

In [74]:
# Load player info from JSON file

# Path to player info JSON file
player_file = "../Resources/nfl_players.json"

# Load csv file to DataFrame
players_df = pd.read_json(player_file)

# Display DataFrame
players_df.head()

,player_id,name,position,height,weight,current_team,birth_date,birth_place,death_date,college,high_school,draft_team,draft_round,draft_position,draft_year,current_salary,hof_induction_year
0,1809,Robert Blackmon,DB,6-0,208.0,None,1967-05-12,"Bay City, TX",None,Baylor,"Van Vleck, TX",Seattle Seahawks,2.0,34.0,1990.0,None,NaN
1,23586,Dean Wells,LB,6-3,248.0,None,1970-07-20,"Louisville, KY",None,Kentucky,"Holy Cross, KY",Seattle Seahawks,4.0,85.0,1993.0,None,NaN
2,355,Kiko Alonso,ILB,6-3,238.0,Miami Dolphins,1990-08-14,"Newton, MA",None,Oregon,"Los Gatos, CA",Buffalo Bills,2.0,46.0,2013.0,"1,075,000",NaN
3,18182,Steve Ramsey,QB,6-2,210.0,None,1948-04-22,"Dallas, TX",1999-10-15,North Texas,"W.W. Samuell, TX",New Orleans Saints,5.0,126.0,1970.0,None,NaN
4,16250,Cory Nelms,CB,6-0,195.0,None,1988-02-27,"Neptune, NJ",None,Miami (FL),"Neptune, NJ",None,NaN,NaN,NaN,None,NaN


### Data clean-up & transformation (players)

#### <font color='red'>Franchesca - Add code below</font>

In [61]:
# Cleanup & Transform the players DataFrame

# Copy the Players DataFrame to Players Transformed Dataframe  
players_t_df = players_df.copy()


# Franchesca: Add code here:

# Refer to Clean-up and Transformation code added for superbowl dataframe and add code to perform required clean-up & transformation 
# In the end keep the dataframe with index as id and reatins the columns you wish to keep with column name in lower case
# We will create SQL table matching the DataFrame 


# Display the DataFrame
players_t_df.head()

,player_id,name,position,height,weight,current_team,birth_date,birth_place,death_date,college,high_school,draft_team,draft_round,draft_position,draft_year,current_salary,hof_induction_year
0,1809,Robert Blackmon,DB,6-0,208.0,None,1967-05-12,"Bay City, TX",None,Baylor,"Van Vleck, TX",Seattle Seahawks,2.0,34.0,1990.0,None,NaN
1,23586,Dean Wells,LB,6-3,248.0,None,1970-07-20,"Louisville, KY",None,Kentucky,"Holy Cross, KY",Seattle Seahawks,4.0,85.0,1993.0,None,NaN
2,355,Kiko Alonso,ILB,6-3,238.0,Miami Dolphins,1990-08-14,"Newton, MA",None,Oregon,"Los Gatos, CA",Buffalo Bills,2.0,46.0,2013.0,"1,075,000",NaN
3,18182,Steve Ramsey,QB,6-2,210.0,None,1948-04-22,"Dallas, TX",1999-10-15,North Texas,"W.W. Samuell, TX",New Orleans Saints,5.0,126.0,1970.0,None,NaN
4,16250,Cory Nelms,CB,6-0,195.0,None,1988-02-27,"Neptune, NJ",None,Miami (FL),"Neptune, NJ",None,NaN,NaN,NaN,None,NaN


## Load data from Dataframe(s) to Postgres DB

#### <font color='red'>Sumit /  Maria / Franchesca - Pending - After Dataframe extraction & transofrmation is completed</font>

### Establish connection to database 

In [ ]:
rds_connection_string = db_user+":"+db_password+"@localhost:5432/nfl_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [ ]:
engine.table_names()

### Use pandas to load CSV & API converted Superbowl DataFrame into database

In [ ]:
# Load superbowl data
superbowl_t_df.to_sql(name='superbowl', con=engine, if_exists='append', index=True)

### Use pandas to load JSON converted Players DataFrame into database

In [ ]:
# Load players data
players_t_df.to_sql(name='players', con=engine, if_exists='append', index=True)

### Confirm data has been added by querying the view

In [ ]:
# Read SQL view
pd.read_sql_query('select * from superbowl_player_view', con=engine).head()

#### <font color='red'>Sumit /  Maria / Franchesca - Pending - Add SQL Scripts - After ETL is completed</font>